In [1]:
import uiautomator2 as u2
import random
import time

In [2]:
d = u2.connect()
d.info

{'currentPackageName': 'com.android.systemui',
 'displayHeight': 2232,
 'displayRotation': 0,
 'displaySizeDpX': 360,
 'displaySizeDpY': 780,
 'displayWidth': 1080,
 'productName': 'YAL-AL10',
 'screenOn': False,
 'sdkInt': 29,
 'naturalOrientation': True}

## 锁屏解锁
自动识别息屏、亮屏状态并解锁, 支持无密码、数字、图案

In [3]:
# 华为系
def unlock():
    info = d.info
    if info.get('screenOn') and info.get('currentPackageName') != 'com.android.systemui':
        # 屏幕已点亮且当前包名不是系统UI（意味着已经在主屏幕或者应用内）
        pass
    elif info.get('screenOn') and info.get('currentPackageName') == 'com.android.systemui':
        # 屏幕已点亮且当前包名是系统UI（可能在锁屏界面）
        d.press('recent')  # 可能会唤醒密码输入界面
        # 模拟解锁手势
        d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
        time.sleep(0.1)
        d.click(0.9, 0.5)
    else:
        # 屏幕未点亮
        d.screen_on()
        d.press('recent')
        # 模拟解锁手势
        d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
        time.sleep(0.1)
        d.click(0.9, 0.5)
        


## 随机打开任意数量个应用
支持三方/系统应用类型限定, 更换请在第九行添加参数pm list pack -3 | -s  
开始打开app前会检测目前已在运行应用并在将运行列表内剔除

In [111]:
def open_random_apps():
    while True:
        try:
            max_apps = int(input('请输入最大将运行的app数量: '))
            break
        except ValueError:
            print('请输入整数!')
    package_list_prefix = str(d.shell('pm list package -3')).split('output=')[1].split(', exit_code')[0].strip("'").split('\\n')[:-1]
    package_list = [pkg.replace('package:', '') for pkg in package_list_prefix if pkg]
    running_app_list = d.app_list_running()
    waiting_app_list = [app for app in package_list if app not in running_app_list]
    for i in range(min(max_apps, len(waiting_app_list))):  # 加入min函数防止超出
        waiting_app = random.choice(waiting_app_list)
        d.app_start(waiting_app)
        try:
            pid = d.app_wait(waiting_app, timeout=20)
            if not pid:
                print(f'{waiting_app}没在运行!')
            else:
                print(f'{waiting_app}进程为{pid}')
            waiting_app_list.remove(waiting_app)
        except Exception as e:
             print(f"捕获到错误：{e}")
        finally:
            waiting_app_list.remove(waiting_app)
            
    return 0
    

In [106]:
open_random_apps()

请输入最大将运行的app数量:  5


com.vnet.tech进程为13942
com.tencent.qqmusic进程为14188
cn.edu.zjicm.wordsnet_d进程为14490


BaseError: stat /data/app/com.adobe.psmobile-iN0WuWoX6WyJS4dyk02dow==/base.apk
package:/data/app/com.adobe.psmobile-iN0WuWoX6WyJS4dyk02dow==/split_config.arm64_v8a.apk
package:/data/app/com.adobe.psmobile-iN0WuWoX6WyJS4dyk02dow==/split_config.en.apk
package:/data/app/com.adobe.psmobile-iN0WuWoX6WyJS4dyk02dow==/split_config.xxhdpi.apk
package:/data/app/com.adobe.psmobile-iN0WuWoX6WyJS4dyk02dow==/split_config.zh.apk: no such file or directory

## 关闭所有应用

In [107]:
d.app_stop_all()

['com.huawei.camera',
 'com.google.android.gms',
 'com.tencent.mm',
 'com.android.phone',
 'com.huawei.android.launcher',
 'com.huawei.securityserver',
 'com.adobe.psmobile',
 'com.weawow',
 'com.android.vending',
 'com.baidu.input_huawei',
 'com.huawei.trustspace',
 'com.huawei.himovie',
 'com.huawei.systemserver',
 'com.huawei.ohos.camera',
 'com.huawei.harmonyos.foundation',
 'com.huawei.nearby',
 'com.huawei.recsys',
 'com.alibaba.android.rimet',
 'com.huawei.hiaction',
 'com.huawei.intelligent',
 'com.huawei.lbs',
 'com.huawei.security.privacycenter',
 'myapp.ashuiai',
 'com.android.systemui',
 'com.android.permissioncontroller',
 'com.android.mms',
 'com.huawei.hwpanpayservice',
 'com.huawei.iaware',
 'com.android.bluetooth',
 'com.eg.android.AlipayGphone',
 'com.tencent.mobileqq',
 'com.huawei.hiview',
 'com.huawei.hiai',
 'com.xunmeng.pinduoduo',
 'com.android.nfc',
 'com.huawei.trustagent',
 'com.huawei.ohos.famanager']

In [109]:
d.app_wait('com.adobe.psmobile')

22702